In [ ]:
%tensorflow_version 1.x
!pip install mock
!pip install tensorflow-gan
!pip install tensorflow-datasets==3.2.1 

%cd /content
! rm -rf gan-tools
! rm -rf stylegan-lowshot
!git clone --single-branch --depth=1 --branch master https://github.com/hannesdm/gan-tools.git
!git clone --single-branch --depth=1 --branch main https://github.com/hannesdm/stylegan-lowshot.git
%cd gan-tools
from keras.datasets import mnist
import impl
from impl import *
from core import vis
from core import gan
from core import constraint
import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['axes.grid'] = False


## Load the cifar10 data
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.  <br/>
**Exercise** We will select a single class of this dataset to model. This can be done by setting the **model_class** variable to the corresponding class. <br/>
One cell lower, a few images of the selected class are shown.

In [ ]:
model_class = 1
(X_train_original, Y_train), (_, _) = cifar10.load_data()
X_train_single_class = X_train_original[np.where(np.squeeze(Y_train) == model_class)]
X_train = X_train_single_class / 127.5 - 1.

In [ ]:
grid = vis.image_grid(X_train_single_class[0:20], 5)
plt.imshow(grid)

## Train the DCGAN
<img src="https://i.imgur.com/NFUiEf5.png" width="450"> <br/>
The following code will train a GAN with a working DCGAN architecture. This training can be controlled by the following parameters:


*   **batches**: The number of batches the GAN should train on.
*   **batch_size**: The size of each batch.
*    **plot_interval**: After how many batches the generator should be sampled and the images shown.

The default parameters may be kept. <br/>
Make sure to train the GAN for a sufficient amount of time in order to see realistic samples. At any point, the training may be stopped by clicking on the stop button or on 'interrupt execution' in the runtime menu at the top of the page.<br/> In the same menu, the runtime type should also be changed to 'GPU'. This will speed up the training of the models. <br/>
**Exercise** Comment on the loss and accuracy of the generator and discriminator, shown during training and discuss its stability. Explain this in function of the GAN setting.





In [ ]:
gan = cifar10_dcgan()
gan.train_random_batches(X_train, batches = 20000, batch_size=32, plot_interval = 50)
vis.show_gan_image_predictions(gan, 32)

In [ ]:
# Plot the final loss curves
def moving_average(a, n=10) :
    s = np.cumsum(a, dtype=float)
    s[n:] = s[n:] - s[:-n]
    return s[n - 1:] / n


plt.figure(figsize=(16, 12))
plt.plot(moving_average(gan.d_losses), c="blue", label="D Loss")
plt.plot(moving_average(gan.g_losses), c="red", label="G Loss")
plt.plot(moving_average(gan.d_accs), c="green", label="D Accuracy")
plt.plot(moving_average(gan.g_accs), c="yellow", label="G Accuracy")
plt.legend(loc="upper left")
plt.show()

## Stability in GANs
Sadly, training a GAN is not always easy. <br/>
Stability during training is important for both discriminator and generator to learn. <br/>
Below is a short video (50s) showing the intermediate results of a GAN being trained on mnist. The final result is a phenomenon known as 'mode collapse'. <br/>
<img src='https://i.imgur.com/lG35xDP.gif'>


## Optional: High Quality Image Generation with StyleGAN

The DCGAN model was an important point in the history of generative adversarial networks. However, these models have difficulty with high resolution images and have long been passed by the current state of the art. </br>
State of the art models for high resolution image generation, such as BigGAN and StyleGAN, can generate new images with high fidelity of e.g. 1024x1024 image data sets. The trade-off is that these models can require weeks to train even with the best GPUs and/or TPUs available. </br>
</br>
**Few-shot learning** A special setting for training generative models is low-shot/few-shot learning where one attempts to create a model that generalizes well on as few samples as possible. This setting allows the power of the state of the art models to be demonstrated while still being able to be trained in a reasonable time. </br>

**StyleGAN few-shot**
The following script allows you to train a StyleGAN with differentiable augmentations on your own data. Few-shot models work best with uniform, clean data where the object takes up the majority of the image.</br>
To the left of Google Colab, click the folder icon on the sidebar, create a new folder in the file explorer and upload your images into it. It's recommended to have at least 100 images. Next, replace the placeholder in the command below with the link to your uploaded folder (e.g. /content/mydata) and execute the command. It's recommended to try out different types of data to see what works and what doesn't.</br>
Alternatively, replace the placeholder by the name of one of the pre-existing data sets:

*   100-shot-obama
*   100-shot-grumpy_cat
*   100-shot-panda
*   100-shot-bridge_of_sighs
*   100-shot-temple_of_heaven
*   100-shot-wuzhen

The script will output intermediate images while training. More full quality samples can be found in the /content/stylegan-lowshot/results folder. Take note that it can take multiple hours before reasonable images start to be generated even when working with these very small data sets.






In [ ]:
%cd /content/stylegan-lowshot
%run run_low_shot.py --dataset=/content/mydata --num-gpus=1 --resolution=128 --show-samples-every=1